# Phân Tích Toàn Diện LBPH Model

Notebook này thực hiện phân tích và trực quan hóa toàn bộ kết quả training và đánh giá của mô hình LBPH.

## Nội dung:
1. **Model Configuration**: Cấu hình mô hình LBPH
2. **Threshold Analysis**: Phân tích ngưỡng quyết định
3. **Confidence Distribution**: Phân bố độ tin cậy
4. **Evaluation Metrics**: Độ chính xác và Coverage
5. **Confusion Matrix**: Ma trận nhầm lẫn
6. **Predictions Analysis**: Phân tích kết quả dự đoán
7. **Comprehensive Summary**: Báo cáo tổng hợp

## 1. Setup và Import

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from IPython.display import display, Markdown, Image
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

LOGS_DIR = Path('../logs/LBHP')
print(f"Logs directory: {LOGS_DIR.absolute()}")
print(f"\nCác file có sẵn:")
for f in sorted(LOGS_DIR.glob('*')):
    if f.is_file():
        print(f"  - {f.name} ({f.stat().st_size / 1024:.1f} KB)")
    else:
        print(f"  - {f.name}/ (directory)")

## 2. Load Dữ Liệu

In [ ]:
with open(LOGS_DIR / 'metadata.json', 'r') as f:
    metadata = json.load(f)

with open(LOGS_DIR / 'lbph_evaluation_report.json', 'r') as f:
    eval_report = json.load(f)

with open(LOGS_DIR / 'confidence_stats.json', 'r') as f:
    confidence_stats = json.load(f)

with open(LOGS_DIR / 'threshold_history.json', 'r') as f:
    threshold_history = json.load(f)

predictions_df = pd.read_csv(LOGS_DIR / 'lbph_predictions.csv')
threshold_results_df = pd.read_csv(LOGS_DIR / 'lbph_threshold_results.csv')

print("Data loaded successfully!")
print(f"Number of classes: {metadata['num_classes']}")
print(f"Train images: {metadata['train_images']}")
print(f"Val images: {metadata['val_images']}")
print(f"Test images: {metadata['test_images']}")

---
# PHẦN 1: CẤU HÌNH MÔ HÌNH
---

## 3. Model Configuration

In [ ]:
print('='*70)
print('                    LBPH MODEL CONFIGURATION')
print('='*70)
params = metadata['model_params']
print(f'\n  Model: Local Binary Pattern Histogram (LBPH)')
print(f'  Method: {eval_report["method"]}')
print(f'\n  LBPH Parameters:')
print(f'    - Radius: {params["radius"]}')
print(f'    - Neighbors: {params["neighbors"]}')
print(f'    - Grid X: {params["grid_x"]}')
print(f'    - Grid Y: {params["grid_y"]}')
print(f'\n  Dataset:')
print(f'    - Number of Classes: {metadata["num_classes"]}')
print(f'    - Train Images: {metadata["train_images"]}')
print(f'    - Validation Images: {metadata["val_images"]}')
print(f'    - Test Images: {metadata["test_images"]}')
print(f'\n  Optimal Threshold: {eval_report["optimal_threshold"]}')
print(f'  Max Images Per Identity: {eval_report["max_images_per_identity"]}')
print('='*70)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

param_names = ['Radius', 'Neighbors', 'Grid X', 'Grid Y']
param_values = [params['radius'], params['neighbors'], params['grid_x'], params['grid_y']]
colors = ['#3498db', '#2ecc71', '#e74c3c', '#9b59b6']

bars = ax.bar(param_names, param_values, color=colors, edgecolor='black', linewidth=1.5)
ax.set_ylabel('Value')
ax.set_title('LBPH Model Parameters', fontweight='bold', fontsize=14)

for bar, val in zip(bars, param_values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2, str(val),
            ha='center', va='bottom', fontweight='bold', fontsize=12)

ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig(LOGS_DIR / 'model_params_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

---
# PHẦN 2: PHÂN TÍCH NGƯỠNG
---

## 4. Threshold Analysis

In [ ]:
th_df = pd.DataFrame(threshold_history)
print("Threshold History:")
display(th_df)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

ax1 = axes[0, 0]
ax1.plot(th_df['threshold'], th_df['accuracy']*100, 'b-o', lw=2, markersize=8, label='Accuracy')
ax1.axvline(x=eval_report['optimal_threshold'], color='red', linestyle='--', lw=2, label=f'Optimal ({eval_report["optimal_threshold"]})')
ax1.set_xlabel('Threshold')
ax1.set_ylabel('Accuracy (%)')
ax1.set_title('Accuracy vs Threshold', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2 = axes[0, 1]
ax2.plot(th_df['threshold'], th_df['coverage']*100, 'g-s', lw=2, markersize=8, label='Coverage')
ax2.axvline(x=eval_report['optimal_threshold'], color='red', linestyle='--', lw=2, label=f'Optimal ({eval_report["optimal_threshold"]})')
ax2.set_xlabel('Threshold')
ax2.set_ylabel('Coverage (%)')
ax2.set_title('Coverage vs Threshold', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

ax3 = axes[1, 0]
ax3.plot(th_df['threshold'], th_df['accuracy']*100, 'b-o', lw=2, markersize=6, label='Accuracy')
ax3_twin = ax3.twinx()
ax3_twin.plot(th_df['threshold'], th_df['coverage']*100, 'g-s', lw=2, markersize=6, label='Coverage')
ax3.axvline(x=eval_report['optimal_threshold'], color='red', linestyle='--', lw=2, alpha=0.7)
ax3.set_xlabel('Threshold')
ax3.set_ylabel('Accuracy (%)', color='blue')
ax3_twin.set_ylabel('Coverage (%)', color='green')
ax3.set_title('Accuracy vs Coverage Trade-off', fontweight='bold')
ax3.tick_params(axis='y', labelcolor='blue')
ax3_twin.tick_params(axis='y', labelcolor='green')
ax3.grid(True, alpha=0.3)

ax4 = axes[1, 1]
ax4.plot(th_df['threshold'], th_df['score']*100, 'm-^', lw=2, markersize=8, label='Score (Acc × Coverage)')
optimal_idx = th_df['score'].idxmax()
ax4.scatter([th_df.loc[optimal_idx, 'threshold']], [th_df.loc[optimal_idx, 'score']*100], 
            color='red', s=200, zorder=5, marker='*', label=f'Best Score')
ax4.set_xlabel('Threshold')
ax4.set_ylabel('Score (%)')
ax4.set_title('Combined Score vs Threshold', fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(LOGS_DIR / 'threshold_analysis_comprehensive.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nOptimal Threshold Analysis:")
print(f"  Best Threshold: {eval_report['optimal_threshold']}")
print(f"  Best Score: {th_df['score'].max()*100:.2f}%")

## 5. Threshold Results Comparison

In [ ]:
eval_th_df = pd.DataFrame(eval_report['threshold_results'])
print("Evaluation Threshold Results:")
display(eval_th_df)

best_idx = eval_th_df['score'].idxmax()
print(f"\nBest configuration:")
print(f"  Threshold: {eval_th_df.loc[best_idx, 'threshold']}")
print(f"  Accuracy: {eval_th_df.loc[best_idx, 'accuracy']*100:.2f}%")
print(f"  Coverage: {eval_th_df.loc[best_idx, 'coverage']*100:.2f}%")
print(f"  Score: {eval_th_df.loc[best_idx, 'score']*100:.2f}%")

---
# PHẦN 3: PHÂN BỐ ĐỘ TIN CẬY
---

## 6. Confidence Distribution Analysis

In [ ]:
print('='*70)
print('                 CONFIDENCE STATISTICS')
print('='*70)
print(f'\n  CORRECT Predictions:')
print(f'    - Count: {confidence_stats["correct"]["count"]}')
print(f'    - Mean Distance: {confidence_stats["correct"]["mean"]:.2f}')
print(f'    - Std: {confidence_stats["correct"]["std"]:.2f}')
print(f'    - Min: {confidence_stats["correct"]["min"]:.2f}')
print(f'    - Max: {confidence_stats["correct"]["max"]:.2f}')
print(f'    - Median: {confidence_stats["correct"]["median"]:.2f}')
print(f'\n  INCORRECT Predictions:')
print(f'    - Count: {confidence_stats["incorrect"]["count"]}')
print(f'    - Mean Distance: {confidence_stats["incorrect"]["mean"]:.2f}')
print(f'    - Std: {confidence_stats["incorrect"]["std"]:.2f}')
print(f'    - Min: {confidence_stats["incorrect"]["min"]:.2f}')
print(f'    - Max: {confidence_stats["incorrect"]["max"]:.2f}')
print(f'    - Median: {confidence_stats["incorrect"]["median"]:.2f}')
print(f'\n  Threshold Used: {confidence_stats["threshold"]}')
print('='*70)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

ax1 = axes[0]
cm = confidence_stats['correct']['mean']
cs = confidence_stats['correct']['std']
im = confidence_stats['incorrect']['mean']
ist = confidence_stats['incorrect']['std']

x = np.linspace(0, 150, 500)
yc = (1/(cs*np.sqrt(2*np.pi))) * np.exp(-0.5*((x-cm)/cs)**2)
yi = (1/(ist*np.sqrt(2*np.pi))) * np.exp(-0.5*((x-im)/ist)**2)

ax1.fill_between(x, yc, alpha=0.5, color='green', label=f'Correct (μ={cm:.1f})')
ax1.fill_between(x, yi, alpha=0.5, color='red', label=f'Incorrect (μ={im:.1f})')
ax1.axvline(x=confidence_stats['threshold'], color='black', linestyle='--', lw=2, label=f'Threshold ({confidence_stats["threshold"]})')
ax1.set_xlabel('Distance')
ax1.set_ylabel('Density')
ax1.set_title('Distance Distribution (Gaussian Approximation)', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2 = axes[1]
categories = ['Correct', 'Incorrect']
means = [cm, im]
stds = [cs, ist]
colors_bar = ['#2ecc71', '#e74c3c']
bars = ax2.bar(categories, means, yerr=stds, color=colors_bar, edgecolor='black', 
               capsize=10, linewidth=1.5, alpha=0.8)
ax2.axhline(y=confidence_stats['threshold'], color='black', linestyle='--', lw=2, 
            label=f'Threshold ({confidence_stats["threshold"]})')
ax2.set_ylabel('Mean Distance')
ax2.set_title('Mean Distance by Prediction Type', fontweight='bold')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

for bar, val, std in zip(bars, means, stds):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 2, f'{val:.1f}',
             ha='center', va='bottom', fontweight='bold', fontsize=11)

ax3 = axes[2]
counts = [confidence_stats['correct']['count'], confidence_stats['incorrect']['count']]
explode = (0.02, 0.02)
ax3.pie(counts, explode=explode, labels=['Correct', 'Incorrect'], colors=colors_bar,
        autopct='%1.1f%%', shadow=True, startangle=90, textprops={'fontsize': 11})
ax3.set_title('Prediction Distribution', fontweight='bold')

plt.tight_layout()
plt.savefig(LOGS_DIR / 'confidence_analysis_comprehensive.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Existing Confidence Distribution Plots

In [ ]:
plots_dir = LOGS_DIR / 'plots'
if plots_dir.exists():
    print("Existing Visualization Plots:")
    for img_path in sorted(plots_dir.glob('*.png')):
        print(f"\n--- {img_path.name} ---")
        display(Image(filename=str(img_path), width=700))

---
# PHẦN 4: ĐÁNH GIÁ MÔ HÌNH
---

## 8. Evaluation Metrics Summary

In [ ]:
metrics = eval_report['metrics']
timing = eval_report['timing']

print('='*70)
print('                    LBPH EVALUATION SUMMARY')
print('='*70)
print(f'\n  Model: {eval_report["model"]}')
print(f'  Optimal Threshold: {eval_report["optimal_threshold"]}')
print(f'  Evaluation Timestamp: {eval_report["timestamp"]}')
print('\n' + '-'*70)
print('  TEST METRICS:')
print('-'*70)
print(f'  Test Accuracy: {metrics["test_accuracy"]*100:.2f}%')
print(f'  Test Coverage: {metrics["test_coverage"]*100:.2f}%')
print(f'  Used Samples: {metrics["test_used_samples"]}/{metrics["test_total_samples"]}')
print('\n' + '-'*70)
print('  TIMING:')
print('-'*70)
print(f'  Threshold Search: {timing["threshold_search_seconds"]:.2f} seconds')
print(f'  Test Evaluation: {timing["test_eval_seconds"]:.2f} seconds')
print('='*70)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

ax1 = axes[0]
metric_names = ['Accuracy', 'Coverage']
metric_values = [metrics['test_accuracy']*100, metrics['test_coverage']*100]
colors = ['#3498db', '#2ecc71']
bars = ax1.bar(metric_names, metric_values, color=colors, edgecolor='black', linewidth=1.5)
ax1.set_ylim([0, 105])
ax1.set_ylabel('Value (%)')
ax1.set_title('Test Metrics', fontweight='bold')
for bar, val in zip(bars, metric_values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, f'{val:.2f}%',
             ha='center', va='bottom', fontweight='bold', fontsize=11)
ax1.grid(axis='y', alpha=0.3)

ax2 = axes[1]
sizes = [metrics['test_used_samples'], metrics['test_total_samples'] - metrics['test_used_samples']]
labels = ['Used', 'Rejected']
colors_pie = ['#2ecc71', '#e74c3c']
explode = (0.02, 0.05)
ax2.pie(sizes, explode=explode, labels=labels, colors=colors_pie, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 11})
ax2.set_title(f'Sample Coverage ({metrics["test_used_samples"]}/{metrics["test_total_samples"]})', fontweight='bold')

ax3 = axes[2]
categories = ['Accuracy', 'Coverage', 'Score']
score = metrics['test_accuracy'] * metrics['test_coverage']
values = [
    metrics['test_accuracy'],
    metrics['test_coverage'],
    score
]
angles = np.linspace(0, 2*np.pi, len(categories), endpoint=False).tolist()
values_radar = values + [values[0]]
angles += angles[:1]
ax3 = plt.subplot(133, polar=True)
ax3.plot(angles, values_radar, 'o-', linewidth=2, color='#3498db')
ax3.fill(angles, values_radar, alpha=0.25, color='#3498db')
ax3.set_xticks(angles[:-1])
ax3.set_xticklabels(categories)
ax3.set_ylim(0, 1)
ax3.set_title('Performance Radar', fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig(LOGS_DIR / 'metrics_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Confusion Matrix

In [ ]:
cm_img_path = LOGS_DIR / 'lbph_confusion_matrix.png'
if cm_img_path.exists():
    print("Confusion Matrix (Test Set):")
    display(Image(filename=str(cm_img_path), width=700))
else:
    print("Confusion matrix image not found.")

cm_val_path = LOGS_DIR / 'plots' / 'confusion_matrix_val.png'
if cm_val_path.exists():
    print("\nConfusion Matrix (Validation Set):")
    display(Image(filename=str(cm_val_path), width=700))

## 10. Threshold Analysis Visualization

In [ ]:
th_img_path = LOGS_DIR / 'lbph_threshold_analysis.png'
if th_img_path.exists():
    print("Threshold Analysis:")
    display(Image(filename=str(th_img_path), width=700))
else:
    print("Threshold analysis image not found.")

---
# PHẦN 5: PHÂN TÍCH DỰ ĐOÁN
---

## 11. Predictions Analysis

In [ ]:
print(f"Predictions DataFrame Shape: {predictions_df.shape}")
print(f"\nColumns: {list(predictions_df.columns)}")
print(f"\nFirst 10 rows:")
display(predictions_df.head(10))

print(f"\nDataFrame Statistics:")
display(predictions_df.describe())

In [ ]:
if 'distance' in predictions_df.columns or 'confidence' in predictions_df.columns:
    dist_col = 'distance' if 'distance' in predictions_df.columns else 'confidence'
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    ax1 = axes[0]
    ax1.hist(predictions_df[dist_col], bins=50, color='#3498db', edgecolor='black', alpha=0.7)
    ax1.axvline(x=predictions_df[dist_col].mean(), color='red', linestyle='--',
                label=f'Mean: {predictions_df[dist_col].mean():.2f}')
    ax1.axvline(x=eval_report['optimal_threshold'], color='green', linestyle='--',
                label=f'Threshold: {eval_report["optimal_threshold"]}')
    ax1.set_xlabel('Distance/Confidence')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Distribution of Prediction Distances', fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    if 'correct' in predictions_df.columns:
        ax2 = axes[1]
        correct_vals = predictions_df[predictions_df['correct'] == True][dist_col]
        incorrect_vals = predictions_df[predictions_df['correct'] == False][dist_col]
        ax2.hist(correct_vals, bins=30, color='green', alpha=0.6, label=f'Correct (n={len(correct_vals)})')
        ax2.hist(incorrect_vals, bins=30, color='red', alpha=0.6, label=f'Incorrect (n={len(incorrect_vals)})')
        ax2.axvline(x=eval_report['optimal_threshold'], color='black', linestyle='--',
                    label=f'Threshold: {eval_report["optimal_threshold"]}')
        ax2.set_xlabel('Distance/Confidence')
        ax2.set_ylabel('Frequency')
        ax2.set_title('Distance by Prediction Correctness', fontweight='bold')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(LOGS_DIR / 'predictions_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("Distance column not found in predictions.")

---
# PHẦN 6: BÁO CÁO TỔNG HỢP
---

## 12. Comprehensive Summary Report

In [ ]:
summary = {
    'model': 'LBPH',
    'params': metadata['model_params'],
    'num_classes': metadata['num_classes'],
    'train_images': metadata['train_images'],
    'val_images': metadata['val_images'],
    'test_images': metadata['test_images'],
    'optimal_threshold': eval_report['optimal_threshold'],
    'test_accuracy': metrics['test_accuracy'] * 100,
    'test_coverage': metrics['test_coverage'] * 100,
    'combined_score': metrics['test_accuracy'] * metrics['test_coverage'] * 100,
    'correct_mean_distance': confidence_stats['correct']['mean'],
    'incorrect_mean_distance': confidence_stats['incorrect']['mean'],
    'distance_separation': confidence_stats['incorrect']['mean'] - confidence_stats['correct']['mean']
}

with open(LOGS_DIR / 'comprehensive_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print('='*70)
print('                 COMPREHENSIVE SUMMARY REPORT')
print('='*70)
print(json.dumps(summary, indent=2))
print('='*70)
print(f'\nSummary saved to: {LOGS_DIR / "comprehensive_summary.json"}')

## 13. Performance Assessment

In [ ]:
print('='*70)
print('                    PERFORMANCE ASSESSMENT')
print('='*70)

print('\n1. RECOGNITION PERFORMANCE:')
acc = metrics['test_accuracy'] * 100
if acc >= 50:
    print(f'   [GOOD] Test Accuracy: {acc:.2f}%')
elif acc >= 20:
    print(f'   [FAIR] Test Accuracy: {acc:.2f}%')
else:
    print(f'   [NEEDS IMPROVEMENT] Test Accuracy: {acc:.2f}%')
    print(f'   Note: Low accuracy is expected for LBPH with large number of classes ({metadata["num_classes"]})')

print('\n2. COVERAGE:')
cov = metrics['test_coverage'] * 100
if cov >= 90:
    print(f'   [EXCELLENT] Coverage: {cov:.2f}%')
elif cov >= 70:
    print(f'   [GOOD] Coverage: {cov:.2f}%')
else:
    print(f'   [NEEDS IMPROVEMENT] Coverage: {cov:.2f}%')

print('\n3. DISTANCE SEPARATION:')
separation = summary['distance_separation']
if separation >= 30:
    print(f'   [EXCELLENT] Distance Separation: {separation:.2f}')
    print(f'   (Good separation between correct and incorrect predictions)')
elif separation >= 15:
    print(f'   [GOOD] Distance Separation: {separation:.2f}')
else:
    print(f'   [FAIR] Distance Separation: {separation:.2f}')
    print(f'   (Overlapping distributions may cause confusion)')

print('\n4. MODEL CHARACTERISTICS:')
print(f'   - LBPH is a traditional computer vision method (non-deep learning)')
print(f'   - Works well for smaller datasets with controlled conditions')
print(f'   - Limited scalability with large number of classes')
print(f'   - Fast training and inference')
print(f'   - No GPU required')

print('\n5. OVERALL ASSESSMENT:')
overall_score = summary['combined_score']
print(f'   Combined Score (Acc × Coverage): {overall_score:.2f}%')
print('='*70)

## 14. Comparison Notes

In [ ]:
print('='*70)
print('            LBPH vs DEEP LEARNING MODELS')
print('='*70)
print('''
| Aspect              | LBPH                        | FaceNet/ArcFace           |
|---------------------|-----------------------------|--------------------------|
| Type                | Traditional CV              | Deep Learning            |
| Training Speed      | Very Fast                   | Slow (GPU required)      |
| Inference Speed     | Fast                        | Moderate                 |
| Scalability         | Limited                     | Excellent                |
| Large Datasets      | Poor performance            | Excellent performance    |
| Feature Learning    | Handcrafted (LBP)           | Learned embeddings       |
| Lighting Variations | Sensitive                   | Robust                   |
| Pose Variations     | Sensitive                   | Robust                   |
| Hardware Required   | CPU only                    | GPU recommended          |
''')
print('='*70)